## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Qaanaaq,GL,77.4840,-69.3632,0.37,74,56,6.93,broken clouds
1,1,Ushuaia,AR,-54.8000,-68.3000,58.66,36,75,21.85,broken clouds
2,2,Hasaki,JP,35.7333,140.8333,66.60,86,100,49.68,overcast clouds
3,3,Kapaa,US,22.0752,-159.3190,82.38,74,40,3.00,light rain
4,4,Tuktoyaktuk,CA,69.4541,-133.0374,-26.95,100,72,3.53,broken clouds


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 74
What is the maximum temperature you would like for your trip? 94


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Kapaa,US,22.0752,-159.3190,82.38,74,40,3.00,light rain
6,6,Champerico,GT,14.3000,-91.9167,84.74,73,0,5.53,clear sky
7,7,Georgetown,MY,5.4112,100.3354,80.53,89,20,0.00,few clouds
19,19,Kununurra,AU,-15.7667,128.7333,91.38,43,37,5.75,scattered clouds
22,22,Vaini,TO,-21.2000,-175.2000,82.56,83,40,13.80,scattered clouds
31,31,Micheweni,TZ,-4.9667,39.8333,78.22,84,70,5.70,light rain
32,32,Rikitea,PF,-23.1203,-134.9692,76.50,75,0,20.18,clear sky
35,35,Avarua,CK,-21.2078,-159.7750,80.65,69,75,14.97,broken clouds
45,45,Hobart,AU,-42.8794,147.3294,79.27,49,0,4.00,clear sky
50,50,Buala,SB,-8.1450,159.5921,84.96,68,100,7.18,overcast clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# Drop the NaNs.
#preferred_cities_df.dropna()

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kapaa,US,82.38,light rain,22.0752,-159.3190,
6,Champerico,GT,84.74,clear sky,14.3000,-91.9167,
7,Georgetown,MY,80.53,few clouds,5.4112,100.3354,
19,Kununurra,AU,91.38,scattered clouds,-15.7667,128.7333,
22,Vaini,TO,82.56,scattered clouds,-21.2000,-175.2000,
31,Micheweni,TZ,78.22,light rain,-4.9667,39.8333,
32,Rikitea,PF,76.50,clear sky,-23.1203,-134.9692,
35,Avarua,CK,80.65,broken clouds,-21.2078,-159.7750,
45,Hobart,AU,79.27,clear sky,-42.8794,147.3294,
50,Buala,SB,84.96,overcast clouds,-8.1450,159.5921,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
# 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kapaa,US,82.38,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
6,Champerico,GT,84.74,clear sky,14.3000,-91.9167,"Hotel y restaurante ""El Diamante"""
7,Georgetown,MY,80.53,few clouds,5.4112,100.3354,Cititel Penang
19,Kununurra,AU,91.38,scattered clouds,-15.7667,128.7333,Hotel Kununurra
22,Vaini,TO,82.56,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
31,Micheweni,TZ,78.22,light rain,-4.9667,39.8333,
32,Rikitea,PF,76.50,clear sky,-23.1203,-134.9692,People ThankYou
35,Avarua,CK,80.65,broken clouds,-21.2078,-159.7750,Paradise Inn
45,Hobart,AU,79.27,clear sky,-42.8794,147.3294,Mantra on Collins Hobart
50,Buala,SB,84.96,overcast clouds,-8.1450,159.5921,Maringe Lagoon Lodge


In [12]:
# 7. Drop the rows where there is no Hotel Name.
# 4a. Determine if there are any empty rows.
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [13]:
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)

In [14]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kapaa,US,82.38,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
6,Champerico,GT,84.74,clear sky,14.3000,-91.9167,"Hotel y restaurante ""El Diamante"""
7,Georgetown,MY,80.53,few clouds,5.4112,100.3354,Cititel Penang
19,Kununurra,AU,91.38,scattered clouds,-15.7667,128.7333,Hotel Kununurra
22,Vaini,TO,82.56,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
32,Rikitea,PF,76.50,clear sky,-23.1203,-134.9692,People ThankYou
35,Avarua,CK,80.65,broken clouds,-21.2078,-159.7750,Paradise Inn
45,Hobart,AU,79.27,clear sky,-42.8794,147.3294,Mantra on Collins Hobart
50,Buala,SB,84.96,overcast clouds,-8.1450,159.5921,Maringe Lagoon Lodge
51,New Norfolk,AU,78.93,broken clouds,-42.7826,147.0587,The Shingles Riverside Cottages


In [21]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))